In [ ]:
import json
import numpy as np
from random import shuffle
from itertools import product

In [ ]:
def randomize_block():
    
    # Randomize condition order
    pitches = [n for n in range(6)]
    ioi_bins = [n for n in range(5)]
    conditions = [c for c in product(pitches, ioi_bins)]
    shuffle(conditions)
    
    # Rearrange trials as needed, so that no adjacent trials share the same pitch or IOI
    trials = [None for _ in range(30)]
    trials[0] = conditions.pop(0)
    for i in range(1, 30):
        j = 0
        while (conditions[j][0] == trials[i-1][0]) or (conditions[j][1] == trials[i-1][1]):
            j += 1
            if j == len(conditions): 
                return None
        trials[i] = conditions.pop(j)
        
    return trials


def combine_blocks(block1, block2, block3):
    
    # Reorganize condition assignments into a single blocks x trials x IV matrix
    megablock = np.full((3, 30, 4), -1, dtype=int)
    for i, block in enumerate((block1, block2, block3)):
        megablock[i, :, 0] = [x[0] for x in block]
        megablock[i, :, 1] = [x[1] for x in block]
    
    return megablock

        
def randomize_ioi(megablock):

    # For each frequency/ioi-bin pair, randomly assign the three sub-iois to the three blocks
    ioi = [0, 1, 2]
    for i in range(6):
        for j in range(5):
            cond_locs = np.where((megablock[:, :, 0] == i) & (megablock[:, :, 1] == j))
            for k in (2, 3):  # INNER LOOP ADDED TO RANDOMIZE LOUDNESS
                shuffle(ioi)
                megablock[cond_locs[0][0], cond_locs[1][0], k] = ioi[0]
                megablock[cond_locs[0][1], cond_locs[1][1], k] = ioi[1]
                megablock[cond_locs[0][2], cond_locs[1][2], k] = ioi[2]

    return megablock


def convert_conditions(megablock):
    
    octave_map = {
        0: 2,
        1: 3,
        2: 4,
        3: 5,
        4: 6,
        5: 7
    }
    
    ioi_map = {
        0: 1000,
        1: 918,
        2: 843,
        3: 774,
        4: 710,
        5: 652,
        6: 599,
        7: 550,
        8: 504,
        9: 463,
        10: 425,
        11: 390,
        12: 358,
        13: 329,
        14: 302
    }
    
    loudness_map = {
        0: 0,
        1: 1,
        2: 2
    }
    
    # Convert 2-dimensional bin x subIOI into IOI ID numbers
    megablock = megablock.copy()
    megablock[:, :, 1] = 3 * megablock[:, :, 1] + megablock[:, :, 2]
    
    # Convert frequency, IOI, and loudness ID numbers to their corresponding values
    megablock[:, :, 0] = np.array([octave_map[x] for x in megablock[:, :, 0].flatten()]).reshape((3, 30))
    megablock[:, :, 1] = np.array([ioi_map[x] for x in megablock[:, :, 1].flatten()]).reshape((3, 30))
    megablock[:, :, 3] = np.array([loudness_map[x] for x in megablock[:, :, 3].flatten()]).reshape((3, 30))
    
    return megablock[:, :, [0, 1, 3]]

# Generate Trial Schedules

In [ ]:
NSESS = 300  # Number of sessions per group

sessions = []
for i in range(NSESS):
    
    # Tone type conditions require the even numbered blocks to contain a copy of each trial types and the odd numbered blocks to contain a copy of each trial type
    # To accomplish this, we'll generate two sessions of the previous experiment and interleave their blocks
    session = np.zeros((6, 30, 3), dtype=int)
    for j in range(2):  
        # Generate three valid blocks for the session
        blocks = []
        while len(blocks) < 3:
            new_block = randomize_block()
            if new_block is not None:
                blocks.append(new_block)
            
        # Reorganize blocks into a single blocks x trials x variable matrix 
        halfsess = combine_blocks(blocks[0], blocks[1], blocks[2])
        # For each frequency/ioi-bin pair, randomly assign the three sub-iois to the three blocks
        halfsess = randomize_ioi(halfsess)
        # Replace frequency and IOI ID numbers with their corresponding octave and milisecond values
        halfsess = convert_conditions(halfsess)
        # Insert blocks into even or odd positions
        session[(j, j+2, j+4), :, :] = halfsess
        
    sessions.append(session)

In [ ]:
# Shuffle order of sessions and save
shuffle(sessions)
for i, session in enumerate(sessions):
    with open('../schedules/session%i.json' % (i+1), 'w') as f:
        json.dump(session.tolist(), f)

# Sanity Checks

In [ ]:
s = np.zeros((6, 30), int)
for k, sess in enumerate(sessions):
    for i, block in enumerate(sess):
        for j, trial in enumerate(block):
            s[i, j] = int(''.join([str(x) for x in trial[:2]]))

    # There should be 90 combinations of IOI and pitch
    if np.unique(s).shape[0] != 90:
        raise Exception('Not all combinations of IOI and pitch are represented in session %i!' % k)

    # All 90 combinations should appear in both the even and odd blocks separately
    if np.unique(s[(0, 2, 4), :]).shape[0] != 90 or np.unique(s[(1, 3, 5), :]).shape[0] != 90:
        raise Exception('Not all trial types appear under each tone type in session %i!' % k)

print('Tests successfully passed!')